In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

1. Обработка пропусков данных

In [2]:
titanic_data = pd.read_csv("train.csv")
titanic_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
titanic_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
titanic_data.drop('Cabin', axis=1,inplace=True)

Столбец Cabin содержит номера кают, но в большинстве случаев там пропущены значения (слишком много NaN). Поэтому вместо заполнения пропусков мы полностью удаляем этот столбец, так как он мало полезен для анализа.

In [5]:
# Заполнение пропусков в "Age" медианным возрастом по классу
median_age_by_class = titanic_data.groupby("Pclass")["Age"].median()

for pclass in range(1, 4):
    titanic_data.loc[(titanic_data["Age"].isnull()) & (titanic_data["Pclass"] == pclass), "Age"] = median_age_by_class[pclass]


Столбец Age содержит пропущенные значения (NaN). Чтобы их заполнить, мы вычисляем медианный возраст (median()) пассажиров для каждого класса (Pclass):

1 класс (пассажиры платили больше – обычно старше)
2 класс (средний возраст)
3 класс (самые дешёвые билеты – чаще молодёжь)
Затем мы заменяем пропуски в "Age", используя медианные значения для соответствующего класса пассажира

In [6]:
# Заполнение пропусков в "Embarked" наиболее частым значением
most_frequent_embarked = titanic_data["Embarked"].mode()[0]
titanic_data.loc[titanic_data["Embarked"].isnull(), "Embarked"] = most_frequent_embarked


In [7]:
titanic_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

2. кодирование категориальных признаков;

Столбец Embarked содержит текстовые данные (категориальные значения), такие как "S", "C", "Q".
Чтобы их преобразовать в числовой формат, мы используем One-Hot Encoding (pd.get_dummies()), который создаёт отдельные столбцы для каждого значения

In [8]:
embarked_dummies = pd.get_dummies(titanic_data['Embarked'], prefix='Embarked')
titanic_data_oh = pd.concat([titanic_data, embarked_dummies], axis=1)
titanic_data_oh.drop('Embarked', axis=1, inplace=True)


In [9]:
titanic_data_oh

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,False,False,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,True,False,False
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,False,False,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,False,False,True
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,False,False,True
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,False,False,True
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,24.0,1,2,W./C. 6607,23.4500,False,False,True
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,True,False,False


После этого удаляем оригинальный столбец Embarked, так как его уже заменили.

2. Label Encoding для "Sex"

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
titanic_data_oh['Sex'] = le.fit_transform(titanic_data_oh['Sex'])


Пол (Sex) представлен строками "male" и "female", которые нельзя передать в модель.
Label Encoding преобразует их в числа:

"male" → 1
"female" → 0

In [11]:
titanic_data_oh

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,False,False,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,True,False,False
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,False,False,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,False,False,True
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,False,False,True
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.0000,False,False,True
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,24.0,1,2,W./C. 6607,23.4500,False,False,True
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,True,False,False


3. Масштабирование данных.

📌 1. Использование MinMaxScaler для "Age"

In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
titanic_data_oh['Age_scaled'] = scaler.fit_transform(titanic_data_oh[['Age']])


MinMaxScaler преобразует значения в диапазон от 0 до 1, используя формулу


In [13]:
titanic_data_oh

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked_C,Embarked_Q,Embarked_S,Age_scaled
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,False,False,True,0.271174
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,True,False,False,0.472229
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,False,False,True,0.321438
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,False,False,True,0.434531
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,False,False,True,0.434531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,False,False,True,0.334004
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.0000,False,False,True,0.233476
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,24.0,1,2,W./C. 6607,23.4500,False,False,True,0.296306
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,True,False,False,0.321438


📌 2. Использование StandardScaler для "Fare"

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
titanic_data_oh['Fare_scaled'] = scaler.fit_transform(titanic_data_oh[['Fare']])


StandardScaler выполняет Z-нормализацию, приводя данные к нулевому среднему и единичному стандартному отклонению.

Z-нормализация (или стандартизация) — это метод преобразования данных таким образом, чтобы:

Среднее значение (μ) стало 0
Стандартное отклонение (σ) стало 1

In [15]:
titanic_data_oh

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked_C,Embarked_Q,Embarked_S,Age_scaled,Fare_scaled
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,False,False,True,0.271174,-0.502445
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,True,False,False,0.472229,0.786845
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,False,False,True,0.321438,-0.488854
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,False,False,True,0.434531,0.420730
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,False,False,True,0.434531,-0.486337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,False,False,True,0.334004,-0.386671
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.0000,False,False,True,0.233476,-0.044381
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,24.0,1,2,W./C. 6607,23.4500,False,False,True,0.296306,-0.176263
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,True,False,False,0.321438,-0.044381
